# Consuming a REST API

In this example, we're going to use the [https://www.geojs.io/](https://www.geojs.io/) REST API.

It is simple, is not rate limited, and does not require auth - that way we can focus on the pydantic side of things.

In [1]:
import requests
from pydantic import BaseModel, ConfigDict, Field, field_validator, ValidationError, IPvAnyAddress

Typically, when we use Pydantic models to read data returned from an API, we don't need to do a lot of validation - the expectation is that the API is consistent and returns what it says it does.

Of course, your mileage may vary, and depending on the API you are using, you might need to be more careful about defining your models. And yes, unfortunately, there are plenty poorly design APIs out in the wild! So, you may end up having to write a lot of custom validators just to make the data you are receiving from the API consistent and easily used in your application.

## Endpoint

In particular, we'll look at the `Geo` endpoints for this API. Given an IP address (or none, in which case it will use your public IP address), it returns geographic information about the ip address.

This is a `get` request, so we do not have to model a JSON payload, just one for the response.

Unfortunately, they do not provide a JSON schema (that I could find), so we're going to have to create our pydantic model from scratch.

Since I'm not interested in all the fields returned by this endpoint, I'm going to create a model with `extra="ignore"`.

Furthermore, this API does not use camel casing for field names, so we won't even have to worry about aliases.

The documentation does not state whether response fields are nullable or not.

The way I would approach this is to make non-nullable the fields which I absolutely need to be populated in order for my program to work, and make the rest nullable, dealing with null values in code.

The docs do state that `organization_name` will be returned as the string `Unknown` when that field value is unknown - I don't like using special values to indicate null/None, so I'm going to use a validator (an after validator, since I do want Pydantic to at least validate/coerce a string value), and simply replace `Unknown` with `None` (making the field nullable of course).

In [2]:
class IPGeo(BaseModel):
    model_config = ConfigDict(extra="ignore")

    ip: IPvAnyAddress
    country: str | None = None
    country_code: str | None = Field(default=None, min_length=2, max_length=2)
    country_code3: str | None = Field(default=None, min_length=2, max_length=3)
    city: str | None = None
    region: str | None = None
    timezone: str | None= None
    organization_name: str | None = None

    @field_validator("organization_name", mode="after")
    @classmethod
    def set_unknown_to_none(cls, value: str):
        if value.casefold() == "unknown":
            return None
        return value

In [3]:
IPGeo(ip="8.8.8.8", country="test", country_code3 = "USA", organization_name="Unknown")

IPGeo(ip=IPv4Address('8.8.8.8'), country='test', country_code=None, country_code3='USA', city=None, region=None, timezone=None, organization_name=None)

Now let's try it when calling the endpoint. Let's specify an IP address first:

In [4]:
url_query = "https://get.geojs.io/v1/geo/{ip_address}.json"

In [5]:
url = url_query.format(ip_address="8.8.8.8")
response = requests.get(url)
response.raise_for_status()

In [6]:
response_json= response.json()

In [7]:
print(response.json())

{'country': 'United States', 'country_code': 'US', 'country_code3': 'USA', 'continent_code': 'NA', 'asn': 15169, 'organization_name': 'GOOGLE', 'organization': 'AS15169 GOOGLE', 'timezone': 'America/Chicago', 'accuracy': 1000, 'ip': '8.8.8.8', 'longitude': '-97.822', 'latitude': '37.751', 'area_code': '0'}


Now, to deserialize this data, the `requests` library already deserializes the JSON string to a Pytrhon dictionary, so we use `model_validate()` not `model_validate_json()`.

In [8]:
data = IPGeo.model_validate(response.json())

In [9]:
data

IPGeo(ip=IPv4Address('8.8.8.8'), country='United States', country_code='US', country_code3='USA', city=None, region=None, timezone='America/Chicago', organization_name='GOOGLE')

And one more:

In [10]:
url = url_query.format(ip_address="23.62.177.155")
response = requests.get(url)
response.raise_for_status()
data = IPGeo.model_validate(response.json())
print(data.model_dump_json(indent=2))

{
  "ip": "23.62.177.155",
  "country": "United States",
  "country_code": "US",
  "country_code3": "USA",
  "city": "El Segundo",
  "region": "California",
  "timezone": "America/Los_Angeles",
  "organization_name": "AKAMAI-AS"
}


We can also query for our own public IP (GeoJS will figure it out):

```python
url = "https://get.geojs.io/v1/ip/geo.json"
response = requests.get(url)
response.raise_for_status
data = IPGeo.model_validate(response.json())
print(data.model_dump_json(indent=2))
```

I am not going to run it, since I don't want to disclose my public IP address :-) 